# 💥 Primeros pasos con ``Python`` 💥

|| <a style="background-color: #1bf8ff; color: #000000; border: none; padding: 10px 15px; border-radius: 5px;">Introducción a Python</a> ||
<a style="background-color: #1bf8ff; color: #000000; border: none; padding: 10px 15px; border-radius: 5px;">Proyecto</a> ||
<a style="background-color: #1bf8ff; color: #000000; border: none; padding: 10px 15px; border-radius: 5px;">Cursos PyM</a> ||
<a style="background-color: #1bf8ff; color: #000000; border: none; padding: 10px 15px; border-radius: 5px;">Luis Fernando Apáez Álvarez</a> ||


---


## 📚 Descripción

El proyecto consta en simular un día de ventas en una papelería que cuenta con 10 sucursales a lo largo de la CDMX. Para ello

1. Crearemos un código para generar esos datos simulados, con lo cual utilizaremos:
* Listas, diccionarios, manejo de f-Strings, bucles y condicionales. Manejo de fechas.
* Librería Pandas, Random, Datetime y Sqlite3.

2. Automatizaremos el código utilizando programación orientada a objetos, donde montaremos el flujo de datos y tendremos control sobre él.
3. Generaremos el insumo de excel para pasárselo a Power BI y crear un tablero asociado para visualizar la información generada. 

### Muestra

In [1]:
from Base import tools as t

# Creamos la bbdd y la tabla de ventas
t.definiciones()

Error: table Ventas_2024 already exists


In [4]:
t.proceso('2024-01-02')

Generación exitosa al 2024-01-02
Inserción existosa
Fecha: 2024-01-02 || Tiempo de ejecución: 0.73 minutos


In [5]:
t.comprobar_fechas()

        FECHA
0  2024-01-01
1  2024-01-02


In [2]:
t.proceso('2024-01-03', '2024-01-05')

Generación exitosa al 2024-01-03
Inserción existosa
Fecha: 2024-01-03 || Tiempo de ejecución: 0.72 minutos
-----------------------------------
Generación exitosa al 2024-01-04
Inserción existosa
Fecha: 2024-01-04 || Tiempo de ejecución: 0.56 minutos
-----------------------------------
Generación exitosa al 2024-01-05
Inserción existosa
Fecha: 2024-01-05 || Tiempo de ejecución: 0.15 minutos
-----------------------------------
Tiempo de ejecución total: 1.43


In [3]:
t.comprobar_fechas()

        FECHA
0  2024-01-01
1  2024-01-02
2  2024-01-03
3  2024-01-04
4  2024-01-05


---

## �� Sentencias SQL

1. Consultas: Select, Limit, Group By, Order By, Alias
2. Vistas, Distinct, Where, Concatenacion

In [6]:
import sqlite3 as sql
import pandas as pd

# Creamos la tabla
query = """
SELECT * FROM Ventas_2024
"""

# Creamos la base de datos, la cual se llamara Ejemplo
conn = sql.connect("Ventas.db")
df = pd.read_sql_query(query, conn)
conn.close()

df.head()

,PRODUCTO,CLAVE_PRODUCTO,PRECIO_VENTA,CANTIDAD_VENDIDO,SUCURSAL,FECHA
0,Mochilas,BKO042,3997.37,1,Cuemanco,2024-01-01
1,Tablets,WRG373,502.82,9,Milpa Alta,2024-01-01
2,Baterías,HDT052,96.86,0,Xochimilco,2024-01-01
3,Pinceles,QTA475,1476.53,1,Polanco,2024-01-01
4,Pinceles,QTA475,1476.53,3,CU,2024-01-01


In [19]:
# Creamos la tabla
query = """
SELECT
    SUCURSAL,
    SUM(TOTAL) AS TOTAL_VENDIDO
FROM
    --- Subconsultas
    (SELECT 
        PRODUCTO,
        CLAVE_PRODUCTO,
        PRECIO_VENTA,
        CANTIDAD_VENDIDO,
        SUCURSAL,
        FECHA,
        --- Podemos agregar una columna nueva
        PRECIO_VENTA * CANTIDAD_VENDIDO AS TOTAL
    FROM 
        Ventas_2024)
--- Agrupaciones
GROUP BY 
    SUCURSAL
--- Ordenamientos
ORDER BY
    TOTAL_VENDIDO DESC
--- Limitar registros
LIMIT 
    5
"""

# Creamos la base de datos, la cual se llamara Ejemplo
conn = sql.connect("Ventas.db")
df = pd.read_sql_query(query, conn)
conn.close()

df

,SUCURSAL,TOTAL_VENDIDO
0,Coapa,18880458.38
1,Narvarte,18427611.49
2,Xochimilco,18402785.81
3,Centro,18338741.68
4,Milpa Alta,18299628.36


In [20]:
# Creamos vista: tablas temporales
query = """
CREATE VIEW Ventas_2024_t
AS
  SELECT 
    PRODUCTO,
    CLAVE_PRODUCTO,
    PRECIO_VENTA,
    CANTIDAD_VENDIDO,
    SUCURSAL,
    FECHA,
    --- Podemos agregar una columna nueva
    PRECIO_VENTA * CANTIDAD_VENDIDO AS TOTAL
  FROM 
    Ventas_2024
"""

conn = sql.connect("Ventas.db")
cur = conn.cursor()
cur.execute(query)
conn.commit()
conn.close()

In [23]:
# Elementos diferentes de una columna
query = """
SELECT
    DISTINCT SUCURSAL
FROM
    Ventas_2024_t
"""

# Creamos la base de datos, la cual se llamara Ejemplo
conn = sql.connect("Ventas.db")
df = pd.read_sql_query(query, conn)
conn.close()

df

,SUCURSAL
0,Cuemanco
1,Milpa Alta
2,Xochimilco
3,Polanco
4,CU
5,Narvarte
6,Zócalo
7,Centro
8,Santa Fé
9,Coapa


In [42]:
# Elementos diferentes de una columna
query = """
SELECT
   PRODUCTO || '_' || CLAVE_PRODUCTO AS LLAVE,
   CLAVE_PRODUCTO,
   FECHA, 
   TOTAL
FROM
    Ventas_2024_t
WHERE
    SUCURSAL = 'Cuemanco'
AND
    TOTAL >= 15000
LIMIT 
    5
"""

# Creamos la base de datos, la cual se llamara Ejemplo
conn = sql.connect("Ventas.db")
df = pd.read_sql_query(query, conn)
conn.close()

df

,LLAVE,CLAVE_PRODUCTO,FECHA,TOTAL
0,Cajas_KIC193,KIC193,2024-01-01,26375.16
1,Mochilas_BKO042,BKO042,2024-01-01,23984.22
2,Cajas_KIC193,KIC193,2024-01-01,22607.28
3,Mochilas_BKO042,BKO042,2024-01-01,35976.33
4,Teclados_OGA612,OGA612,2024-01-01,16366.02


In [38]:
# Elementos diferentes de una columna
query = """
SELECT
    PRODUCTO
    SUCURSAL,
    MAX(PRECIO_VENTA) AS MAXIMO
FROM
    Ventas_2024_t
WHERE
    SUCURSAL = 'Cuemanco'
"""

# Creamos la base de datos, la cual se llamara Ejemplo
conn = sql.connect("Ventas.db")
df = pd.read_sql_query(query, conn)
conn.close()

df

,SUCURSAL,MAXIMO
0,Mochilas,3997.37
